In [10]:
import pandas as pd
import os
import re

In [11]:
fires = pd.DataFrame()
pathname = '../data'

for filename in os.listdir(pathname):
    if os.path.isfile(os.path.join(pathname, filename)) and 'Confirmed' in filename:
        if os.path.splitext(filename)[1] == '.csv':
            fires = pd.concat([fires, pd.read_csv(os.path.join(pathname, filename))], ignore_index=True)

fires = fires.drop('OBJECTID', axis=1).drop_duplicates()

In [12]:
def strip_time(entry):
    m = re.match('(\d{4}/\d{2}/\d{2})', entry)
    return m.group(1)

In [13]:
fires['confirmed_fire'] = True

In [14]:
fires['alarm_datetime'] = fires.apply(lambda row: strip_time(row.alm_date) + ' ' + row.alm_time, axis=1)
fires['clear_datetime'] = fires.apply(lambda row: strip_time(row.clr_date) + ' ' + row.clr_time, axis=1)

In [15]:
fires['alarm_datetime'] = pd.to_datetime(fires['alarm_datetime'])
fires['clear_datetime'] = pd.to_datetime(fires['clear_datetime'])

In [16]:
fires = fires.drop(['alm_time', 'clr_time'], axis=1)

In [17]:
fires = fires.sort_values('inci_no')
fires

,X,Y,FID,inci_no,descript,alm_date,station,number_,street,st_type,...,clr_date,alarms,complete,LastUpdateDate,StartDate,EndDate,number,confirmed_fire,alarm_datetime,clear_datetime
1249,-93.286230,45.027240,1.0,12-0000000,HazMat release investigation w/no HazMat ...,2012/02/21 00:00:00,20,NaN,- 4801 Interstate 94 Eb,,...,2012/02/21 00:00:00,1,1,2015/03/11 11:13:29,NaN,NaN,4800,True,2012-02-21 14:45:03,2012-02-21 14:58:30
1250,-93.298301,44.997412,2.0,12-0000001,EMS-Asst medics-did NOT provide any BLS Treatm...,2012/01/01 00:00:00,04,NaN,18,AVE,...,2012/01/01 00:00:00,1,1,2015/03/11 11:13:29,NaN,NaN,1401,True,2012-01-01 00:02:51,2012-01-01 00:17:20
1251,-93.298333,44.948333,3.0,12-0000003,"EMS call, excluding vehicle accident with inju...",2012/01/01 00:00:00,22,NaN,Hennepin,AVE,...,2012/01/01 00:00:00,1,1,2015/03/11 11:13:29,NaN,NaN,3001,True,2012-01-01 00:05:09,2012-01-01 00:30:53
1252,-93.247266,44.963666,4.0,12-0000004,"Medical assist, assist EMS crew ...",2012/01/01 00:00:00,07,NaN,Cedar,AVE,...,2012/01/01 00:00:00,1,1,2015/03/11 11:13:29,NaN,NaN,917,True,2012-01-01 00:05:58,2012-01-01 00:15:37
1253,-93.311575,45.006027,5.0,12-0000006,Dispatched & cancelled en route ...,2012/01/01 00:00:00,14,NaN,26,AVE,...,2012/01/01 00:00:00,1,1,2015/03/11 11:13:29,NaN,NaN,,True,2012-01-01 00:10:53,2012-01-01 00:14:57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40853,-93.308313,45.022283,NaN,22-0015322,Road freight or transport vehicle fire ...,2022/04/18 00:00:00+00,14,NaN,Penn,AVE,...,2022/04/18 00:00:00+00,1,1,2022/04/19 18:30:38+00,2022/04/18 10:31:57+00,2022/04/18 11:18:47+00,3701,True,2022-04-18 10:31:57,2022-04-18 11:18:47
41065,-93.272734,44.933889,NaN,22-0015443,"Outside rubbish, trash or waste fire ...",2022/04/19 00:00:00+00,17,NaN,3,AVE,...,2022/04/19 00:00:00+00,1,1,2022/04/20 18:30:37+00,2022/04/19 09:02:55+00,2022/04/19 09:37:48+00,3814,True,2022-04-19 09:02:55,2022-04-19 09:37:48
41066,-93.295626,45.022366,NaN,22-0015457,"Outside rubbish, trash or waste fire ...",2022/04/19 00:00:00+00,14,NaN,Fremont,AVE,...,2022/04/19 00:00:00+00,1,1,2022/04/20 18:30:37+00,2022/04/19 11:49:26+00,2022/04/19 12:19:04+00,3711,True,2022-04-19 11:49:26,2022-04-19 12:19:04
41067,-93.291397,44.999175,NaN,22-0015459,Dumpster or other outside trash receptacle fir...,2022/04/19 00:00:00+00,04,NaN,Broadway,,...,2022/04/19 00:00:00+00,1,1,2022/04/20 18:30:37+00,2022/04/19 12:31:35+00,2022/04/19 12:37:24+00,916,True,2022-04-19 12:31:35,2022-04-19 12:37:24


In [20]:
years = fires['alarm_datetime'].dt.year.unique()
years

array([2012, 2013, 2014, 2015, 2021, 2022])